In [23]:
import numpy as np
from scipy.stats import norm


## Problem 1

Let $S$ be the column vector with components $S^{[1]}$, $S^{[2]}$, where the stock prices $S^{[j]}$ have risk-neutral dynamics
$$\mathrm{d}S_t^{[j]}=rS_t^{[j]}\mathrm{d}t+\sigma_{[j]}S_t^{[j]}\mathrm{d}W_t^{[j]}\quad j=1,2$$
with risk-free interest rate $r = 0.05$, and constant volatilities $\sigma_{[1]} = 0.3$, $\sigma_{[2]} = 0.2$.
The time-0 prices are $S_0^{[1]} = 100$, $S_0^{[2]} = 110$. The P-Brownian motions $W^{[1]}$ and $W^{[2]}$ have correlation $\rho = 0.8$.


`1a`

Question:

Let $X$ be the column vector with components $X^{[1]}$, $X^{[2]}$ where $X^{[j]}:= \log S^{[j]}$. Find the covariance matrix of $X_T$.

Hint: One approach is to manually fill in the covariance matrix, using relationships such as
$Cov(W_T^{[1]}, W_T^{[2]}) = 0.8T$ in combination with the volatilities.

Another approach is to use matrix multiplication: write $X_T$ as a nonrandom vector plus $\Sigma W_T$ where $\Sigma$ is the nonrandom diagonal matrix with diagonal elements $\sigma_{[1]}, \sigma_{[2]}$, and $W$ is the random column vector with components $W^{[1]}, W^{[2]}$. Then $Cov(X_T ) = \mathbb{E}(\Sigma X_T X_T^\top ) = \mathbb{E}(\Sigma W_T W_T^\top \Sigma^\top) = \Sigma Cov(W_T)\Sigma^\top = T \Sigma Corr(W_T)\Sigma^\top$.

Answer:

- $R_t=r S_t^{[j]}$
- $A_t=\sigma_{[j]} S_t^{[j]}$
- $\partial_t X^{[j]} = 0$
- $\partial_s X^{[j]} = \frac{1}{S^{[j]}}$
- $\partial_{ss} X^{[j]} = -\frac{1}{(S^{[j]})^2}$

By Ito’s formula - 
$$\begin{aligned}
dX^{[j]}(S_t, t) &= \left[ \partial_t X^{[j]}(S_t, t) + R_t \partial_{s} X^{[j]}(S_t, t) + \frac{A_t^2}{2} \partial_{ss} X^{[j]}(S_t, t) \right]dt + A_t \partial_{s} X^{[j]}(S_t, t) dW_t \\
dX^{[j]}(S_t, t) &= \left[ r S_t^{[j]} \frac{1}{S^{[j]}} - \frac{(\sigma_{[j]} S_t^{[j]})^2}{2} \cdot \frac{1}{(S^{[j]})^2}\right]dt + \sigma_{[j]} S_t^{[j]}  \frac{1}{S^{[j]}} dW_t \\
\end{aligned}$$

$$dX^{[j]}(S_t, t) = \left[ r - \frac{\sigma_{[j]}^2}{2} \right]dt + \sigma_{[j]} dW_t $$
Expand it,
$$dX^{[1]}(S_t, t) = \left[ r - \frac{\sigma_{[1]}^2}{2} \right]dt + \sigma_{[1]} dW_t $$
$$dX^{[2]}(S_t, t) = \left[ r - \frac{\sigma_{[2]}^2}{2} \right]dt + \sigma_{[2]} dW_t $$

Let $M$ be the covariance matrix of $X_T$, then
$$M = Cov(X_T, X_T) = \mathbb{E}[X_T X_T^\top] = \mathbb{E}\begin{pmatrix}
(X_T^{[1]})^2 & X_T^{[1]}X_T^{[2]} \\
X_T^{[1]}X_T^{[2]} & (X_T^{[2]})^2
\end{pmatrix} = 
\begin{pmatrix}
\sigma_{[1]}^2 (W_T^{[1]})^2 & \sigma_{[1]}\sigma_{[2]} W_T^{[1]}W_T^{[2]} \\
\sigma_{[1]}\sigma_{[2]} W_T^{[1]}W_T^{[2]} & \sigma_{[2]}^2(W_T^{[2]})^2
\end{pmatrix} =
\begin{pmatrix}
\sigma_{[1]}^2 T & \sigma_{[1]}\sigma_{[2]}\rho T\\
\sigma_{[1]}\sigma_{[2]} \rho T & \sigma_{[2]}^2 T
\end{pmatrix} 

$$
Since only the terms from two $dW_t$'s multiplying together would survive.

Consider a basket $H := \frac12S^{[1]} + \frac12S^{[2]}$ of one-half of a share of each stock.

`1b`

Question:

Using 10000 standard Monte Carlo simulations, estimate the time-0 price $C$ of an option that pays $(H_T − 110)^+$ at time $T = 1.0$. Also give the standard error [the sample standard deviation, divided by the square root of the number of simulations] of your Monte Carlo estimate.

You may either use a random number generator that produces normals with a given covariance matrix (which you found in (a)), or alternatively use a random number generator that produces independent normals which you then transform to introduce correlation.

In either approach, each of the 10000 simulations should use just one $\mathbb{R}^2$-valued random vector $Z$ of simulated normal zero-mean random variables.

Answer:



### Monte Carlo Algorithm - ordinary
Let $Y$ be the discounted payoff of the basket $H$ consisting of one-half of a share of each stock $S^{[j]}$, $H = \frac12S^{[1]} + \frac12S^{[2]}$.

In our case, 
$$
\begin{aligned}X_t&=\log(S_t)\\
dX_t &= \left[ r - \frac{\sigma^2}{2} \right]dt + \sigma dW_t
\end{aligned}
$$

In our algorithm, since $X_T - X_0$ is known to be multivariate normal, and the option payoff depends only on $X_T$, it is not necessary to simulate the entire path. Below is our algorithm:
- Initialize the $m$-th Monte Carlo simulation:
$$X_0^{[j]} = \log(S_0^{[j]})$$
- Each stock has time dynamics of geometric Brownian motion:
$$dX_t^{[j]} = \left[ r - \frac{\sigma_{[j]}^2}{2} \right]dt + \sigma_{[j]} dW_t^{[j]}$$
- Final log-stock price: 
   - $$X_T^{[j]} = X_0^{[j]} + \left(r - \frac{\sigma_{[j]}^2}{2}\right) T+\left(L\sqrt{T}Z\right)^{[j]} $$ 
   - where $Z$ is IID standard normal, $LL^\top=M$ and $MT$ is the covariance matrix for $X_T$.
   - Dimension of $X_T$ is $D\times 1$, $X_0$ is $D\times 1$, $\mu T$ is $D\times 1$, $L\sqrt{T}$ is $D\times D$, $Z$ is $D\times 1$.
- Final stock price: 
$$S_{T}^{[j]} = \exp(X_T^{[j]})$$
- Discounted call price: 
$$Y = e^{-rT}(H_{T} - K)^+ =e^{-rT}\left(\frac12 S_{T}^{[1]} + \frac12 S_{T}^{[2]} - K\right)^+ $$
- Repeat this $M$ times
- Monte Carlo estimate of the time-0 call price: 
$$\hat C_M:=\frac{Y_1+Y_2+\cdots+Y_M}M$$

Implementation note:
- `numpy.random.Generator.multivariate` normal generates $LZ$, which will be our approach below
- `numpy.linalg.cholesky` returns $L$ given $M$

In [24]:
class MultiGBM:

    def __init__(self, S0, r, correlations, sigma):
        self.S0 = S0
        self.r = r
        self.correlations = correlations
        self.sigma = sigma

In [25]:
hw6p1dynamics = MultiGBM(S0=np.array([100, 110]),
                         r=0.05,
                         correlations=np.array([[1, 0.8], [0.8, 1]]),
                         sigma=np.diag([0.3, 0.2]))

In [26]:
class CallOnBasket:

    def __init__(self, K, T, weights):
        self.K = K
        self.T = T
        self.weights = weights

In [27]:
hw6p1contract=CallOnBasket(K=110, T=1.0, weights = np.array([1/2, 1/2]))

In [28]:
class MCengine:

    def __init__(self, M, antithetic, control, seed):
        self.M = M                                  # How many simulations
        self.antithetic = antithetic
        self.control = control
        self.rng = np.random.default_rng(seed=seed) # Seeding the random number generator with a specified number helps make the calculations reproducible

    def price_callonbasket_multiGBM(MC, contract, dynamics):
        # You complete the coding of this function.
        # self.rng.multivariate_normal may be useful.
        # See documentation for numpy.random.Generator.multivariate_normal
        # as self.rng is an instance of numpy.random.Generator

        # You are not required to support the case where MC.control = MC.antithetic = True
        # (simultaneous use of control variate and antithetic)
        # But you are required to support the other 3 possible settings of MC.antithetic/MC.control
        # namely False/False, True/False, False/True.
        # (ordinary MC, antithetic without control, control without antithetic)

        S0 = dynamics.S0
        r = dynamics.r
        sigma = dynamics.sigma
        T = contract.T
        correlations = dynamics.correlations
        weights = contract.weights
        K = contract.K

        cov = sigma @ correlations @ sigma
        L = np.linalg.cholesky(cov)

        # Generate normal random variables
        Z = MC.rng.standard_normal(size=(MC.M, len(S0)))

        # Generate asset prices at maturity
        S_T = S0 * np.exp((r - 0.5 * np.diag(sigma) ** 2) * T + (L @ Z.T).T * np.sqrt(T))

        # Calculate basket price at maturity
        basket_prices = np.dot(S_T, weights)

        # Calculate the payoff
        discount_payoffs = np.exp(-r * T) * np.maximum(basket_prices - K, 0)

        if MC.antithetic:
            Z_av = -Z
            S_T_av = S0 * np.exp((r - 0.5 * np.diag(sigma) ** 2) * T + (L @ Z_av.T).T * np.sqrt(T))
            basket_prices_av = np.dot(S_T_av, weights)
            discount_payoffs_av = np.exp(-r * T) * np.maximum(basket_prices_av - K, 0)
        
        if MC.control:
            # Expected mean of control variate, by Black-Scholes formula
            class GBMdynamics:
                def __init__(self, S, r, rGrow, sigma=None):
                    self.S = S
                    self.r = r
                    self.rGrow = rGrow
                    self.sigma = sigma

                def update_sigma(self, sigma):
                    self.sigma = sigma
                    return self
                
            class CallOption:
                def __init__(self, K, T, price=None):
                    self.K = K
                    self.T = T
                    self.price = price

            class AnalyticEngine:
                def __init__(self):
                    pass

                def BSpriceCall(self, dynamics, contract):
                    # ignores contract.price if given, because this function calculates price based on the dynamics

                    F = dynamics.S*np.exp(dynamics.rGrow*contract.T)
                    std = dynamics.sigma*np.sqrt(contract.T)
                    d1 = np.log(F/contract.K)/std+std/2
                    d2 = d1-std
                    return np.exp(-dynamics.r*contract.T)*(F*norm.cdf(d1)-contract.K*norm.cdf(d2))
            
            # Calculate analytical control payoff expectation
            control_analytic = AnalyticEngine()
            control_dynamics = GBMdynamics(S=S0.prod()**0.5, 
                                        rGrow=r - (np.sum(sigma ** 2) - 2 * cov[0][1]) / 8, 
                                        r=r, 
                                        sigma=np.sum(cov) ** 0.5 / 2)
            control_contract = CallOption(K=110, T=1.0)
            control_analytic_payoff = control_analytic.BSpriceCall(control_dynamics, control_contract)
            
            # Calculate control basket price at maturity
            control_basket_prices = np.prod(S_T, axis=1)**0.5

            # Calculate the payoff
            control_discount_payoffs = np.exp(-r * T) * np.maximum(control_basket_prices - K, 0)

            # Calculate the price with control variate
            cov_payoff_control = np.cov(discount_payoffs, control_discount_payoffs)[0, 1]
            var_control = np.var(control_discount_payoffs)
            beta = cov_payoff_control / var_control

            adjusted_payoffs = discount_payoffs + beta * (control_analytic_payoff - control_discount_payoffs)
            discount_payoffs = adjusted_payoffs

        call_price = np.mean(discount_payoffs)
        standard_error = np.std(discount_payoffs) / np.sqrt(MC.M)

        if MC.antithetic:
            call_price_av = np.array([discount_payoffs, discount_payoffs_av]).mean(axis=0)
            call_price = np.mean(call_price_av)
            standard_error = np.std(call_price_av) / np.sqrt(MC.M)


        return(call_price, standard_error)

In [29]:
hw6p1bMC=MCengine(M=10000, antithetic=False, control=False, seed=0)
(call_price_ordinary, std_err_ordinary) = hw6p1bMC.price_callonbasket_multiGBM(hw6p1contract, hw6p1dynamics)
print("Call price, price std, of ordinary MC: ", call_price_ordinary, std_err_ordinary)

Call price, price std, of ordinary MC:  9.983294077510442 0.16496212753829412


`1c`

Question:

Use 10000 antithetic pairs $(Z,−Z)$ to estimate $C$, together with a standard error (L5.30).

### Monte Carlo Algorithm - antithetic variates
Let $Y$ be the discounted payoff of the basket $H$ consisting of one-half of a share of each stock $S^{[j]}$, $H = \frac12S^{[1]} + \frac12S^{[2]}$.

In our case, 
$$
\begin{aligned}X_t&=\log(S_t)\\
dX_t &= \left[ r - \frac{\sigma^2}{2} \right]dt + \sigma dW_t
\end{aligned}
$$

In our algorithm, since $X_T - X_0$ is known to be multivariate normal, and the option payoff depends only on $X_T$, it is not necessary to simulate the entire path. Below is our algorithm:
- Initialize the $m$-th Monte Carlo simulation:
$$X_0^{[j]} = \log(S_0^{[j]})$$
- Each stock has time dynamics of geometric Brownian motion:
$$dX_t^{[j]} = \left[ r - \frac{\sigma_{[j]}^2}{2} \right]dt + \sigma_{[j]} dW_t^{[j]}$$
- Generate antithetic-variate:
$$Z, \tilde{Z}$$
- Final log-stock price: 
   - $$X_T^{[j]} = X_0^{[j]} + \left(r - \frac{\sigma_{[j]}^2}{2}\right) T+\left(L\sqrt{T}Z\right)^{[j]} $$ 
   - $$\tilde{X_T^{[j]}} = X_0^{[j]} + \left(r - \frac{\sigma_{[j]}^2}{2}\right) T+\left(L\sqrt{T}\tilde{Z}\right)^{[j]} $$ 
   - where $Z = -\tilde{Z}$ is IID standard normal, $LL^\top=M$ and $MT$ is the covariance matrix for $X_T$.
   - Dimension of $X_T$ is $D\times 1$, $X_0$ is $D\times 1$, $\mu T$ is $D\times 1$, $L\sqrt{T}$ is $D\times D$, $Z$ is $D\times 1$.
- Final stock price:
$$S_{T}^{[j]} = \exp(X_T^{[j]})$$
$$\tilde{S_T}^{[j]} = \exp(\tilde{X_T}^{[j]})$$
- Discounted call price:
$$\begin{aligned} Y &= e^{-rT}(H_{T} - K)^+ =e^{-rT}\left(\frac12 S_{T}^{[1]} + \frac12 S_{T}^{[2]} - K\right)^+ \\
\tilde{Y} &= e^{-rT}(\tilde{H_{T}} - K)^+ =e^{-rT}\left(\frac12 \tilde{S_T}^{[1]} + \frac12 \tilde{S_T}^{[2]} - K\right)^+ 
\end{aligned}$$
- Average them:
$$Y^{\mathrm{av}} = \frac{Y + \tilde{Y}}{2}$$
- Repeat this $M$ times
- Monte Carlo estimate of the time-0 call price: 
$$\hat C_M:=\frac{Y^{\mathrm{av}}_1+Y^{\mathrm{av}}_2+\cdots+Y^{\mathrm{av}}_M}M$$


In [30]:
hw6p1cMC=MCengine(M=10000,antithetic=True,control=False,seed=0)
(call_price_AV, std_err_AV) = hw6p1cMC.price_callonbasket_multiGBM(hw6p1contract,hw6p1dynamics)
print("Call price, price std, of antithetic variate MC: ", call_price_AV, std_err_AV)

Call price, price std, of antithetic variate MC:  9.907786908965779 0.09448969258409289


Consider the “geometric basket” $G := (S^{[1]}S^{[2]})^{1/2}$.

`1d`

Question:

The random variable $\log G_T$ is normally distributed (because it’s a linear transformation of a multivariate normal vector). Show that $\log G_T$ has expectation
$$\frac12 \log(S_0^{[1]}S_0^{[2]}) + \left( r-\frac{\sigma_{[1]}^2 + \sigma_{[2]}^2}{4}\right)T$$

and variance
$$\frac{\sigma_{[1]}^2 + 2\rho \sigma_{[1]}\sigma_{[2]}+ \sigma_{[2]}^2}{4}T$$

Answer:
$$dX^{[1]} = \left[ r - \frac{\sigma_{[1]}^2}{2} \right]dt + \sigma_{[1]} dW_t $$
$$dX^{[2]} = \left[ r - \frac{\sigma_{[2]}^2}{2} \right]dt + \sigma_{[2]} dW_t $$

$$\begin{aligned}
\mathbb{E}[\log G_T] &= \mathbb{E}[\log(S_T^{[1]}S_T^{[2]})^{1/2}] \\
&= \frac12 \mathbb{E}[\log(S_T^{[1]}) + \log(S_T^{[2]})] \\
&= \frac12 \mathbb{E}\left[\log(S_0^{[1]} + \Delta S_T^{[1]}) + \log(S_0^{[2]}+ \Delta S_T^{[2]})\right] \\
&= \frac12 \log(S_0^{[1]} S_0^{[2]}) + \frac12 \mathbb{E}\left[\log(\Delta S_T^{[1]}) + \log(\Delta S_T^{[2]})\right] \\
&= \frac12 \log(S_0^{[1]} S_0^{[2]}) + \frac12 \mathbb{E}\left[\left(r - \frac{\sigma_{[1]}^2}{2}\right) + \left(r - \frac{\sigma_{[2]}^2}{2}\right)\right]T \\
&= \frac12 \log(S_0^{[1]}S_0^{[2]}) + \left( r-\frac{\sigma_{[1]}^2 + \sigma_{[2]}^2}{4}\right)T
\end{aligned}$$

$$\begin{aligned}
Var[\log G_T] &= Var[\log(S_T^{[1]}S_T^{[2]})^{1/2}] \\
&= Var\left[\frac12 \log(S_T^{[1]}S_T^{[2]})\right] \\
&= \frac14 Var\left[ \log(S_T^{[1]}) + \log(S_T^{[2]})\right] \\
&= \frac14 \left[Var\left[ \log(S_T^{[1]})\right] +2 Cov\left[ \log(S_T^{[1]}) \log(S_T^{[2]})\right] + Var\left[\log(S_T^{[2]})\right]\right] \\
&= \frac14 \left[\sigma_{[1]}^2 T  +2 \rho \sigma_{[1]}\sigma_{[2]}T + \sigma_{[2]}^2 T\right] \\
&= \frac{\sigma_{[1]}^2 + 2\rho \sigma_{[1]}\sigma_{[2]}+ \sigma_{[2]}^2}{4}T
\end{aligned}$$


`1e`

Question:

Let $C^G$ be the time-0 price of a geometric basket option paying $(G_T −K)^+$ at time $T$.

Express $C^G$ in terms of the function $C^{BS}$ defined in FINM 33000 L6. Specifically, fill in the

$$C^G =C^{BS}(\_\_,0,K,T,\_\_,r,\_\_)$$

Your answer should be a general formula, in which you have not substituted 0.8 for $\rho$, etc. (You may also do the substitutions, but don’t neglect the general formula).

Answer:
$$\begin{aligned}
\log X_T&\sim\mathrm{Normal}(\log X_t+(R_{grow}-\sigma^2/2)(T-t), \sigma^2(T-t)) \\
C^G &=C^{BS}(X_t,0,K,T,R_{grow},r,\sigma)
\end{aligned}$$

In our case,
$$\begin{aligned}
t&=0 \\
\log G_T&\sim\mathrm{Normal}\left(\frac12 \log(S_0^{[1]}S_0^{[2]}) + \left( r-\frac{\sigma_{[1]}^2 + \sigma_{[2]}^2}{4}\right)T, \frac{\sigma_{[1]}^2 + 2\rho \sigma_{[1]}\sigma_{[2]}+ \sigma_{[2]}^2}{4}T\right) \\
&= \mathrm{Normal}\left(\log G_0 + \left( r-\frac{\sigma_{[1]}^2 + \sigma_{[2]}^2}{4}\right)T, \frac{\sigma_{[1]}^2 + 2\rho \sigma_{[1]}\sigma_{[2]}+ \sigma_{[2]}^2}{4}T\right) 
\end{aligned}$$

Extracting coefficients,
$$\begin{aligned}
X_t &= \log G_0 \\
R_{grow}-\sigma^2/2 &= r-\frac{\sigma_{[1]}^2 + \sigma_{[2]}^2}{4} \\
\sigma^2 &= \frac{\sigma_{[1]}^2 + 2\rho \sigma_{[1]}\sigma_{[2]}+ \sigma_{[2]}^2}{4} 
\end{aligned}$$

Setting equivalence,
$$\begin{aligned}
R_{grow}  &= r-\frac{\sigma_{[1]}^2 - 2\rho \sigma_{[1]}\sigma_{[2]}+ \sigma_{[2]}^2}{8}  \\

C^G &=C^{BS}\left(G_0,0,K,T,r-\frac{\sigma_{[1]}^2 - 2\rho \sigma_{[1]}\sigma_{[2]}+ \sigma_{[2]}^2}{8},r,\frac{\sqrt{\sigma_{[1]}^2 + 2\rho \sigma_{[1]}\sigma_{[2]}+ \sigma_{[2]}^2}}{2} \right) \\

\end{aligned}$$


`1f`

Question:

Using a geometric basket option as a control variate, run $M = 10000$ Monte Carlo simulations to estimate $C$, together with a standard error. Use the control variate estimate $\hat{C}_M^{\mathrm{cv},\hat{\beta}}$ from L6.7 or L6.8. Use the (asymptotically valid) standard error $\hat{C}_M^{\mathrm{cv},\hat{\beta}}/\sqrt{M}$.

### Monte Carlo Algorithm - control variates
Let $Y$ be the discounted payoff of the basket $H$ consisting of one-half of a share of each stock $S^{[j]}$, $H = \frac12S^{[1]} + \frac12S^{[2]}$. 

In our case, 
$$
\begin{aligned}X_t&=\log(S_t)\\
dX_t &= \left[ r - \frac{\sigma^2}{2} \right]dt + \sigma dW_t
\end{aligned}
$$

In our algorithm, since $X_T - X_0$ is known to be multivariate normal, and the option payoff depends only on $X_T$, it is not necessary to simulate the entire path. Below is our algorithm:
- Initialize the $m$-th Monte Carlo simulation:
$$X_0^{[j]} = \log(S_0^{[j]})$$
- Each stock has time dynamics of geometric Brownian motion:
$$dX_t^{[j]} = \left[ r - \frac{\sigma_{[j]}^2}{2} \right]dt + \sigma_{[j]} dW_t^{[j]}$$
- Consider the original basket, and the "geometric basket" as the control-variate:
$$\begin{aligned} H &:= \frac12S^{[1]} + \frac12S^{[2]} \\
G &:= (S^{[1]}S^{[2]})^{1/2} \end{aligned}$$
- Final log-stock price: 
   - $$X_T^{[j]} = X_0^{[j]} + \left(r - \frac{\sigma_{[j]}^2}{2}\right) T+\left(L\sqrt{T}Z\right)^{[j]} $$ 
   - where $Z$ is IID standard normal, $LL^\top=M$ and $MT$ is the covariance matrix for $X_T$.
   - Dimension of $X_T$ is $D\times 1$, $X_0$ is $D\times 1$, $\mu T$ is $D\times 1$, $L\sqrt{T}$ is $D\times D$, $Z$ is $D\times 1$.
- Final stock price:
$$S_{T}^{[j]} = \exp(X_T^{[j]})$$
- Note that the random variable $\log G_T$ is normally distributed:
$$\log G_T \sim\mathrm{Normal}\left(\frac12 \log(S_0^{[1]}S_0^{[2]}) + \left( r-\frac{\sigma_{[1]}^2 + \sigma_{[2]}^2}{4}\right)T, \frac{\sigma_{[1]}^2 + 2\rho \sigma_{[1]}\sigma_{[2]}+ \sigma_{[2]}^2}{4}T\right)$$
- The expected payoff of the control variate is provided by the Black-Scholes formula:
$$C^* := C^G =C^{BS}\left(G_0,0,K,T,r-\frac{\sigma_{[1]}^2 - 2\rho \sigma_{[1]}\sigma_{[2]}+ \sigma_{[2]}^2}{8},r,\frac{\sqrt{\sigma_{[1]}^2 + 2\rho \sigma_{[1]}\sigma_{[2]}+ \sigma_{[2]}^2}}{2} \right)$$
- Discounted call price for the original, and the control-variate:
$$\begin{aligned} Y &= e^{-rT}(H_{T} - K)^+ =e^{-rT}\left(\frac12S^{[1]} + \frac12S^{[2]} - K\right)^+ \\
Y^* &= e^{-rT}(G_{T} - K)^+ =e^{-rT}\left((S_{T}^{[1]}S_{T}^{[2]})^{1/2} - K\right)^+ \end{aligned}$$
- Repeat this $M$ times
- Monte Carlo estimate of the time-0 call price: 
$$\begin{aligned}\hat{C}_M^{\mathrm{cv},\beta}:=\bar{Y}_M+\beta(C^*-\bar{Y}_M^*) \quad \mathrm{where} \quad\bar{Y}_M&:=\frac1M(Y_1+\cdots+Y_M)\\\quad\quad\bar{Y}_M^*&:=\frac1M(Y_1^*+\cdots+Y_M^*)\end{aligned}$$
where $\beta$ is optimized to be
$$\hat{\beta}:=\frac{\sum_m(Y_m-\bar{Y}_M)(Y_m^*-\bar{Y}_M^*)}{\sum_m(Y_m^*-\bar{Y}_M^*)^2}$$

------
Math note:
- Expectation of control variate estimate:
$$\mathbb{E}\hat{C}_M^{\mathrm{cv},\beta}:=\mathbb{E}\hat{C}_M+\beta(C^*-C^*)=C$$
- Variance of control variate estimate:
$$\begin{aligned}
\mathrm{Var}(\hat{C}_{M}^{\mathrm{cv},\beta})& =\mathrm{Var}(\bar{Y}_M-\beta\bar{Y}_M^*)=\frac1M\mathrm{Var}(Y-\beta Y^*) \\
&=\frac1M\big[\mathrm{Var}(Y)-2\beta\mathrm{Cov}(Y,Y^*)+\beta^2\mathrm{Var}(Y^*)\big]
\end{aligned}$$
- Reduce variance by choosing $\beta$ to minimize this,
$$\beta_{\mathrm{optimal}}=\mathrm{Cov}(Y,Y^*)/\mathrm{Var}(Y^*)$$
- Unfortunately the Cov and Var are usually not both known. But we can use the sample covariance and variance. Let 
$$\hat{\beta}:=\frac{\sum_m(Y_m-\bar{Y}_M)(Y_m^*-\bar{Y}_M^*)}{\sum_m(Y_m^*-\bar{Y}_M^*)^2}$$



In [31]:
hw6p1fMC=MCengine(M=10000, antithetic=False, control=True, seed=0)
(call_price_CV, std_err_CV) = hw6p1fMC.price_callonbasket_multiGBM(hw6p1contract, hw6p1dynamics)
print("Call price, price std, of control variate MC: ", call_price_CV, std_err_CV)

Call price, price std, of control variate MC:  9.991401659760298 0.004430826501668424


## Problem 2

Let the bank account and non-dividend paying stock have risk-neutral dynamics
$$\begin{aligned}
dB_t = rB_t dt, \quad B_0&=1 \\
dS_t = rS_t dt + \sigma S_t dW_t, \quad S_0&>1
\end{aligned}$$
where $\sigma > 0$ and $W$ is a $\mathbb{P}$-Brownian motion.

Consider a $K$-strike $T$-expiry vanilla call option, and let $C$ denote its time-0 price.

`2a`

Question:

Let $S_0 = 100, \sigma = 0.2, r = 0.02, K = 150, T = 1$.

Run 100000 ordinary Monte Carlo simulations to estimate $C$, together with a standard error.

In [32]:
class GBM:

    def __init__(self,sigma,r,S0):
        self.sigma = sigma
        self.r = r
        self.S0 = S0

In [33]:
hw6p2dynamics=GBM(sigma=0.2, r=0.02, S0=100)

In [34]:
class CallOption:

    def __init__(self,K,T):
        self.K=K
        self.T=T

In [35]:
hw6p2contract=CallOption(K=150, T=1)

In [46]:
class MCimportanceEngine:

    def __init__(self, M, lamb, seed):
        self.M = M                                  # How many simulations
        self.lamb = lamb                            # drift adjustment
        self.rng = np.random.default_rng(seed=seed) # Seeding the random number generator with a specified number helps make the calculations reproducible

    def price_call_GBM(self, contract, dynamics):
        # You complete the coding of this function.
        # self.rng.normal may be useful.
        # See documentation for numpy.random.Generator.normal
        # as self.rng is an instance of numpy.random.Generator

        S0 = dynamics.S0
        r = dynamics.r
        sigma = dynamics.sigma
        T = contract.T
        K = contract.K
        lamb = self.lamb

        # Generate normal random variables, in this case W_T^*
        Z = self.rng.normal(loc=0, scale=1, size=(self.M))
        W_star = Z * np.sqrt(T)
        W = W_star + lamb * T

        # Generate asset prices at maturity
        S_T = S0 * np.exp((r - 0.5 * sigma ** 2) * T + sigma * W)

        # Calculate the payoff
        discount_payoffs = np.exp(-r * T) * np.maximum(S_T - K, 0) 

        # Importance Sampling
        z_score = -(lamb * W_star) - (0.5 * lamb** 2 * T)
        fx_gx = np.exp(z_score)
        discount_payoffs *= fx_gx

        call_price = np.mean(discount_payoffs)
        standard_error = np.std(discount_payoffs) / np.sqrt(self.M)

        return(call_price, standard_error)


In [47]:
hw6p2aMC=MCimportanceEngine(M=100000,lamb=0,seed=0) #zero drift adjustment gives ordinary MC

(call_price_ordinary, std_err_ordinary) =  hw6p2aMC.price_call_GBM(hw6p2contract,hw6p2dynamics)
print("Call price, price std, of ordinary MC: ", call_price_ordinary, std_err_ordinary)

Call price, price std, of ordinary MC:  0.2527033283360926 0.0076092552464345755


`2b`

Question:

Suppose that we sample from a new probability measure $\mathbb{P}^*$, under which $W$ now has constant
drift $\lambda$ instead of drift 0. Thus $W_t = W_t^* + \lambda t$ where $W^*$ is a standard $\mathbb{P}^*$-BM. 

Find the $\mathbb{P}^*$-expectation $\mathbb{E}^* S_T$ in terms of $S_0, r, \sigma, \lambda$, and $T$.

Calculate $\lambda$ such that $\mathbb{E}^* S_T = 165$.

(Why did we choose 165? The picture in L6.17 shows that the optimal distribution from which to sample will have a mean that is greater than the strike $K$. So let’s choose 10% higher than $K$. This will not be optimal, but we expect that it will be an improvement over ordinary Monte Carlo. There are more systematic ways to determine a reasonable drift adjustment, not utilized here.)

Answer:

When choosing $\lambda$, it may be too difficult to make $S_T$'s distribution agree entirely with the optimal distribution. So let's just make $S_T$'s mean $\approx$ the optimal distribution's mean. If $S$ is GBM,
$$\mathrm{d}S_t=rS_t\mathrm{d}t+\sigma S_t\mathrm{d}W_t=(r+\sigma\lambda)S_t\mathrm{d}t+\sigma S_t\mathrm{d}W_t^*$$
The solution to this SDE is:
$$S_T=S_0 e^{(r+\sigma\lambda - \frac12 \sigma^2)T + \sigma W_T^*}$$

Taking the $\mathbb{P}^*$-expectation,
$$\mathbb{E}^*S_T=S_0 \mathbb{E}^*\left[\exp{(r+\sigma\lambda - \frac12 \sigma^2)T + \sigma W_T^*}\right]$$
Since $W_T^* \sim \mathcal{N}(0,T)$, $\sigma W_T^* \sim \mathcal{N}(0,\sigma^2 T)$, the moment generating function of a random variable $X\sim \mathcal{N}(\mu,\sigma^2)$ is:
$$\mathbb{E}[e^{tX}]=\exp\left( t\mu + \frac12 t^2 \sigma^2\right)$$

So,
$$\mathbb{E}[\exp(\sigma W_T^*)]=\exp\left(\frac12 \sigma^2 T\right)$$

$$\mathbb{E}^*S_T=S_0 \mathbb{E}^*\left[\exp\left((r+\sigma\lambda - \frac12 \sigma^2)T\right) \exp\left(\frac12 \sigma^2 T \right)\right]
=S_0e^{(r+\sigma\lambda)T}$$

For $\mathbb{E}^*S_T=165$, 
$$\lambda = 2.4039$$

In [48]:
lamb = (np.log(1.65) - 0.02) / 0.2
print(lamb)

2.4038764395624455


`2c`

Question:

Run 100000 importance sampling simulations, using the specific drift adjustment calculated in (b), to estimate $C$, together with a standard error. Be aware that your zero-mean normal random draws, here, simulate increments of $W^*$ not $W$.

Each simulation should require only one number to be generated by `rng.normal`.


>- $\underline{\textbf{Risk-Neutral Dynamics:}}$
>- We will simulate the following PDE, with:
$$\begin{align*}
dB_{t} &= rB_{t}dt \quad\quad\qquad &B_{0} = 1 \\ 
dS_{t} &= rS_{t}dt + \sigma S_{t}dW_{t} \quad &S_{0} > 0 \\
\end{align*}$$
>
>- Let $W_{t} = W^{*}_{t} + \lambda t$ and apply it to the dynamics:
$$\begin{align*}
dW_{t} &= dW^{*}_{t} + \lambda dt \\
dS_{t} &= rS_{t} dt + \sigma S_{t} dW_{t} \\
&= \left(r + \sigma\lambda\right)S_{t} dt + \sigma S_{t} dW^{*}_{t} \\
\end{align*}$$
>
>- Tranform to log form:
$$\begin{align*}
d\log(S_{t}) &= \left(r - \frac{\sigma^{2}}{2}\right)dt + \sigma dW_{t} \\
&= \left(r + \sigma\lambda - \frac{\sigma^{2}}{2} \right)dt + \sigma dW^{*}_{t} \\
\end{align*}$$
>
>- Then we have that, by integrations: 
$$\begin{align*}
S_{t} &= \exp\left(\log\left(S_{t}\right)\right)\\
&= S_{0}\exp\left(\left(r + \sigma\lambda - \frac{\sigma^{2}}{2} \right)t + \sigma W^{*}_{t}\right) \\
\log(S_{t}) &\sim \mathcal{N}\left(\log(S_{0}) + \left(r + \sigma\lambda - \frac{\sigma^{2}}{2} \right)t, \sigma^{2}t\right)
\end{align*}$$
>
>- In this case, if $d\log(S_{t})$ did not have a drift term, then it will behave as:
$$\sigma W_{t} \sim \mathcal{N}\left(0, \sigma^{2}t\right)$$
>
>- NOTE:
>>- We integrated $dW_{t}$ and it did not got to zero.
>>- This is because the we are not taking the expectations, so not considering Martingale properties.
>>- $\mathbb{E}\left[\int W_{t}dt\right] = 0$
>>- $\textbf{If, } W_{t} \sim \mathcal{N}\left(0, t\right) \textbf{, Then, } \Longrightarrow\mathbb{E}\left[W_{t}\right] = 0 \text{ and } \mathrm{Var}\left[W_{t}\right] = t$


>- $\underline{\textbf{Proof: Short Version}}$
>- We have the following useful relation for the expected payout: <br>
$$\textbf{If, } S_{t} \sim \mathcal{N}\left(m, v\right) \textbf{, Then, } \mathbb{E}\left[\exp\left(S_{t}\right)\right] = \exp\left(m + \frac{v}{2}\right)$$
>
>- From the above distribution of $\log(S_{t})$, we get the following 
$$\begin{align*}
\mathbb{E}\left[\exp\left(\log(S_{t})\right)\right] &= \mathbb{E}\left[S_{t}\right] \\
&= S_{0}\exp\left(m + \frac{v}{2}\right) \\
\end{align*}$$
>
>- where, 
$$\begin{align*}
m &= \left(r + \sigma\lambda - \frac{\sigma^{2}}{2}\right)t \\
v &= \left(\sigma^{2}t\right) \\
m + \frac{v}{2} &= \left(r + \sigma\lambda\right)t \\
\end{align*}$$
>
>- The final form is:
$$\begin{align*}\therefore \mathbb{E}\left[S_{t}\right] &= S_{0}\exp\left(m + \frac{v}{2}\right) \\
&= S_{0}\exp\left(\left(r + \sigma\lambda\right)t\right) \\
\end{align*}$$


>- $\underline{\textbf{Proof: Long Version}}$
>- We have the following useful relation for the expected payout:
$$\begin{align*}
\mathbb{E}\left[S_{T}\right] &= \mathbb{E}^{*}\left[S_{0}\exp\left(\left(r - \frac{\sigma^{2}}{2}\right)T + \sigma W_{t}\right)\right] \\
&= \mathbb{E}^{*}\left[S_{0}\exp\left(\left(r - \frac{\sigma^{2}}{2}\right)T + \sigma \left(W^{*}_{t} + \lambda T\right)\right)\right] \\
&= \mathbb{E}^{*}\left[S_{0}\exp\left(\left(r - \frac{\sigma^{2}}{2} + \sigma\lambda\right)T + \sigma W^{*}_{t}\right)\right] \\
&= S_{0}\exp\left(\left(r - \frac{\sigma^{2}}{2} + \sigma\lambda\right)T\right)\mathbb{E}^{*}\left[\exp\left(\sigma W^{*}_{t}\right)\right] \\
&= S_{0}\exp\left(\left(r - \frac{\sigma^{2}}{2} + \sigma\lambda\right)T\right)\exp\left(\frac{\sigma^{2}}{2}T\right) \\
&= S_{0}\exp\left(\left(r + \sigma\lambda\right)T\right)
\end{align*}$$
>
>- The final form is:
$$\therefore \color{red}{\boxed{\color{black}{\mathbb{E}\left[S_{T}\right] = \exp\left(\log(S_{0}) + (r + \sigma\lambda)T\right) = S_{0}\exp\left((r + \sigma\lambda)T\right) = F^{*}}}}$$
>
>- Given, $F^{*}, S_{0}, r, \sigma, T$ , we can solve for the drift constant $\lambda$:
$$\lambda = \frac{\frac{\log\left(\frac{F^{*}}{S_{0}}\right)}{T} - r}{\sigma}$$


In [49]:
hw6p2cMC=MCimportanceEngine(M=100000,lamb=lamb,seed=0) # Fill in the lamb parameter with the lambda that you compute in (b)
(call_price_importsamp, std_err_importsamp) =  hw6p2cMC.price_call_GBM(hw6p2contract,hw6p2dynamics)
print("Call price, price std, of importance sampling MC: ", call_price_importsamp, std_err_importsamp)

Call price, price std, of importance sampling MC:  0.24843662621391358 0.0007734233296681457
